In [ ]:
ls

sample_data/


In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

Saving data_transformations.py to data_transformations.py
Saving ffgsm_.py to ffgsm_.py
Saving fgsm_.py to fgsm_.py
Saving models.py to models.py
Saving no_attack.py to no_attack.py
Saving pgd_.py to pgd_.py
Saving pgdl2_.py to pgdl2_.py
Saving rep_transformations.py to rep_transformations.py
Saving slide_.py to slide_.py
Saving stochastic_attack.py to stochastic_attack.py


In [ ]:
pip install torchattacks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 155 kB 32.5 MB/s 


In [ ]:
#Import libraries

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models
import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

#from torchattacks import PGD, FGSM, FFGSM
import rep_transformations as rt
from no_attack import NO_ATTACK
from stochastic_attack import STOCHASTIC_ATTACK
from fgsm_ import FGSM_
from ffgsm_ import FFGSM_
from slide_ import SLIDE_
from pgd_ import PGD_
from pgdl2_ import PGDL2_
from data_transformations import DctBlurry

In [ ]:
pip install torchattacks


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Show CUDA settings

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {cuda_id}")

cuda_name = torch.cuda.get_device_name(cuda_id)
print(f"Name of current CUDA device: {cuda_name}\n")
print(f"CUDA devide properties: {torch.cuda.get_device_properties(0)}")

Is CUDA supported by this system? True
CUDA version: 11.3
ID of current CUDA device: 0
Name of current CUDA device: Tesla T4

CUDA devide properties: _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
#Downloading Fashion MNIST Dataset
train_list_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #DctBlurry(threshold=5),
    ])

test_list_transforms = transforms.Compose([
        transforms.ToTensor(),
    ])

fashion_mnist_train = dsets.FashionMNIST(root='./data1/',
                          train=True,
                          transform=train_list_transforms,
                          download=True)

fashion_mnist_aux = dsets.FashionMNIST(root='./data1/',
                         train=False,
                         transform=test_list_transforms,
                         download=True)

fashion_mnist_test, fashion_mnist_valid = torch.utils.data.random_split(fashion_mnist_aux, [5000, 5000], generator=torch.Generator().manual_seed(42))

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data1/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data1/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data1/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data1/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data1/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data1/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data1/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data1/FashionMNIST/raw



In [ ]:
#Initialize DataLoader

batch_size = 128
torch.manual_seed(0)
n_workers = 5

train_loader  = torch.utils.data.DataLoader(dataset=fashion_mnist_train,
                                           batch_size=batch_size,
                                           shuffle=42, num_workers=n_workers)

test_loader = torch.utils.data.DataLoader(dataset=fashion_mnist_test,
                                         batch_size=batch_size,
                                         shuffle=42, num_workers=n_workers)

valid_loader = torch.utils.data.DataLoader(dataset=fashion_mnist_valid,
                                         batch_size=batch_size,
                                         shuffle=42, num_workers=n_workers)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
#Initialize transformations

id_transf = rt.Identity()
fft_transf = rt.FFT()
dct_transf = rt.DCT()
jpeg_transf = rt.JPEG()

list_transf = [id_transf, fft_transf, dct_transf, jpeg_transf]

In [ ]:
#Get model and optimizer

def generate_model(pretrained=True, lr=0.001):

    #Init model and optimizer
    model = torchvision.models.resnet50(pretrained=True).cuda() #CNN.cuda() to resnet 50
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    #Change model architecture 
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(2048, 10, bias=True)
    
    #Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        model.to(device)
        
    return model, optimizer

In [ ]:
#modified test function with 3 attacks and toggle between Pixel and DCT representation.Default is Pixel

def get_accuracy(model, test_loader, atk):
    
    model.eval()
    correct = 0
    total = 0    

    for images, labels in test_loader:

        images = atk(images, labels).cuda()              
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()

    return 100 * float(correct) / total

In [ ]:
#train function with PGD attack and toggle between DCT and Pixel representation .Default is Pixel

def train_(model, train_loader, optimizer, loss, atk, num_epochs=5):

    model.train()

    for epoch in range(num_epochs):

        total_batch = len(fashion_mnist_train) // batch_size
        
        for i, (batch_images, batch_labels) in enumerate(train_loader):

           
            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Train. Atk: {atk.attack}, Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [ ]:
def round_robin(model, train_loader, optimizer, loss, atks_list, num_epochs=5):

    model.train() 
    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size

    for epoch in range(num_epochs): #+1 to round up

        count = 0

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            atk = atks_list[count]
            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            count += 1
            count %= num_attacks

            if (i+1) % 100 == 0:
                print(f'RR. Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [ ]:
def greedy(model, train_loader, valid_loader, optimizer, loss, atks_list, num_epochs=5):

    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size

    for epoch in range(num_epochs):

        loss_list = [0]*num_attacks

        model.eval()

        for i, (batch_images, batch_labels) in enumerate(valid_loader):

            Y = batch_labels.cuda()
            current_batch_size = Y.shape[0]

            for i, atk in enumerate(atks_list):

                X = atk(batch_images, batch_labels).cuda()

                pre = model(X)
                cost = loss(pre, Y)

                loss_list[i] += float(cost)/current_batch_size

        max_loss_arg = np.argmax(np.array(loss_list))
        atk = atks_list[max_loss_arg]

        print(f"Greedy. Epoch [{epoch+1}/{num_epochs}], Worst loss attack: {atk.attack}, loss: {loss_list[max_loss_arg]}")

        model.train()

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Greedy. Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [ ]:
def multivariate_probability(model, train_loader, valid_loader, optimizer, loss, atks_list, num_epochs=5):

    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size
    w_list = [1]*num_attacks
    eta = 0.1

    for epoch in range(num_epochs):

        loss_list = [0]*num_attacks
        model.eval()

        for i, (batch_images, batch_labels) in enumerate(valid_loader):

            Y = batch_labels.cuda()
            current_batch_size = Y.shape[0]

            for i, atk in enumerate(atks_list):

                X = atk(batch_images, batch_labels).cuda()

                pre = model(X)
                cost = loss(pre, Y)

                loss_list[i] += float(cost)/current_batch_size

        loss_list = np.array(loss_list)
        total_loss = loss_list.sum()
        prob_list = np.cumsum(loss_list)
        prob_list /= total_loss

        log = f"MP. Epoch [{epoch+1}/{num_epochs}], atks: "
        for i, atk in enumerate(atks_list):
            log += f"({atk.attack},{prob_list[i]}) "

        print(log)

        model.train()

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            rand = random.random()
            atk_idx = 0
            for j in range(1,num_attacks):
                if rand > prob_list[j]:
                    atk_idx = j
                else:
                    break

            atk = atks_list[atk_idx]
            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'MP. Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [ ]:
def get_model_acc(model, test_loader, atks_list):
    
    model_acc = {}
    
    for atk in atks_list:
        model_atk_acc = get_accuracy(model, test_loader, atk)
        model_acc[atk.attack] = model_atk_acc
        
    return model_acc

In [ ]:
#Define attacks_list generation

def generate_atks(model_atk, eps=0.3):
    
    atks_list = []
    atks_list.append(NO_ATTACK(model_atk))
    atks_list.append(STOCHASTIC_ATTACK(model_atk, eps=eps))
    atks_list.append(FGSM_(model_atk, eps=eps))
    atks_list.append(PGD_(model_atk, eps=eps))
    atks_list.append(PGD_(model_atk, eps=eps, transf=dct_transf))
    atks_list.append(PGD_(model_atk, eps=eps, transf=fft_transf))
        
    return atks_list

In [ ]:
#### Auto-encoder
# Creating a PyTorch class
# 28*28 ==> 9 ==> 28*28
class AE(torch.nn.Module):
	def __init__(self):
		super().__init__()
		
		# Building an linear encoder with Linear
		# layer followed by Relu activation function
		# 784 ==> 9
		self.encoder = torch.nn.Sequential(
			torch.nn.Linear(28 * 28, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 64),
			torch.nn.ReLU(),
			torch.nn.Linear(64, 36),
			torch.nn.ReLU(),
			torch.nn.Linear(36, 18),
			torch.nn.ReLU(),
			torch.nn.Linear(18, 9)
		)
		
		# Building an linear decoder with Linear
		# layer followed by Relu activation function
		# The Sigmoid activation function
		# outputs the value between 0 and 1
		# 9 ==> 784
		self.decoder = torch.nn.Sequential(
			torch.nn.Linear(9, 18),
			torch.nn.ReLU(),
			torch.nn.Linear(18, 36),
			torch.nn.ReLU(),
			torch.nn.Linear(36, 64),
			torch.nn.ReLU(),
			torch.nn.Linear(64, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 28 * 28),
			torch.nn.Sigmoid()
		)

	def forward(self, x):
		encoded = self.encoder(x)
		decoded = self.decoder(encoded)
		return decoded

# Model Initialization
model = AE()

# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()

# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(),
							lr = 1e-1,
							weight_decay = 1e-8)

In [ ]:
def adv_outputs(model, test_laoder,atk):
    
    model.eval()
    attacked_img = []  

    for images, labels in test_loader:

        images = atk(images, labels).cuda()              
        attacked_img.append(images)
  
    return attacked_img

In [ ]:
## loss should between origina regenerated - adv regenerated***

In [ ]:
# epoch was 5 changed it to 1 for speed i cant wait until 70
def get_model_avg(n_models=1, procedure="None", num_epochs=1, eps=0.3,test_loader = test_loader):

    loss = nn.CrossEntropyLoss()
    iters_result = {}
    attacked_img = [] # store adverserial image outputs
    for i in range(n_models):
        model, optim = generate_model()
        atks_model = generate_atks(model, eps=eps)

        if iters_result == {}:
            for atk in atks_model:
                iters_result[atk.attack] = []

        cost0 = train_(model, train_loader, optim, loss,
                       atks_model[0], num_epochs=num_epochs)

        if procedure == "RR":
            cost1 = round_robin(model, train_loader, optim,
                                loss, atks_model, num_epochs=num_epochs)
        elif procedure == "Greedy":
            cost1 = greedy(model, train_loader, valid_loader,
                           optim, loss, atks_model, num_epochs=num_epochs)

        elif procedure == "MP":
            cost1 = multivariate_probability(
                model, train_loader, valid_loader, optim, loss, atks_model, num_epochs=num_epochs)
        elif procedure == "PGD":
            atk_pgd = PGD_(model, eps=eps, steps=20)
            attacked_img.append(adv_outputs(model,test_loader,atk_pgd)) # store attacked images
            cost1 = train_(model, train_loader, optim, loss,
                           atk_pgd, num_epochs=num_epochs)
        elif procedure == "PGD_DCT":
            atk_pgd_dct = PGD_(model, eps=eps, steps=20, transf=dct_transf)
            attacked_img.append(adv_outputs(model,test_loader,atk_pgd_dct))# store attacked images
            cost1 = train_(model, train_loader, optim, loss,
                           atk_pgd_dct, num_epochs=num_epochs)
        elif procedure == "PGD_FFT":
            atk_pgd_fft = PGD_(model, eps=eps, steps=20, transf=fft_transf)
            attacked_img.append(adv_outputs(model,test_loader,atk_pgd_fft))# store attacked images
            cost1 = train_(model, train_loader, optim, loss,
                           atk_pgd_fft, num_epochs=num_epochs)

        acc = get_model_acc(model, test_loader, atks_model)

        for (key, value) in acc.items():
            temp = iters_result[key]
            temp.append(value)
            iters_result[key] = temp

        print(f'Model average. Iter: {i+1}/{n_models}.')

    results = {}
    for (key, values) in iters_result.items():
        aux_array = np.array(values)
        mean = np.mean(aux_array)
        std = np.std(aux_array)
        results[key] = str(round(mean, 2))+"+"+str(round(std, 2))
    return results,attacked_img

In [ ]:
#raise Exception('Stop cell')

In [ ]:
#Generate accuracy df/table
std_acc,attacked_im = get_model_avg(n_models=1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Train. Atk: NO_ATTACK, Epoch [1/1], lter [100/468], Loss: 0.48818153142929077
Train. Atk: NO_ATTACK, Epoch [1/1], lter [200/468], Loss: 0.5273450613021851
Train. Atk: NO_ATTACK, Epoch [1/1], lter [300/468], Loss: 0.3148314654827118
Train. Atk: NO_ATTACK, Epoch [1/1], lter [400/468], Loss: 0.3603181540966034
Model average. Iter: 1/1.


In [ ]:
## Pixel based auto encoder
def pixel_autoencoder():
  epochs = 3
  outputs = []
  losses = []
  for epoch in range(epochs):
    for (image, _) in train_loader:
      
    
      # Reshaping the image to (-1, 784)
      image = image.reshape(-1, 28*28)
      
      # Output of Autoencoder
      reconstructed = model(image)
      
      # Calculating the loss function
      loss = loss_function(reconstructed, image)
      
      # The gradients are set to zero,
      # the gradient is computed and stored.
      # .step() performs parameter update
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      # Storing the losses in a list for plotting
      losses.append(loss.detach().numpy())
      outputs.append((epochs, image, reconstructed))
      return outputs,losses

pixel_autoencoder_outputs,pixel_autoencoder_loss = pixel_autoencoder()

In [ ]:
########

torch.Size([784, 8, 16])

#####Confused######
len(pixel_autoencoder_outputs[0])
a = pixel_autoencoder_outputs[0][1].reshape(-1,8,16)
a.shape

In [ ]:
df = pd.DataFrame(columns = list(std_acc.keys()))
df = df.append(std_acc, ignore_index=True)
df

,NO_ATTACK,RAND_ID,FGSM_ID,PGD_ID,PGD_DCT,PGD_FFT
0,85.38+0.0,52.68+0.0,4.02+0.0,0.1+0.0,0.14+0.0,13.32+0.0


In [ ]:
std_acc,attacked_im = get_model_avg(n_models=1, procedure='PGD')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creati

Train. Atk: NO_ATTACK, Epoch [1/1], lter [100/468], Loss: 0.5227192640304565
Train. Atk: NO_ATTACK, Epoch [1/1], lter [200/468], Loss: 0.6321813464164734
Train. Atk: NO_ATTACK, Epoch [1/1], lter [300/468], Loss: 0.4647132456302643
Train. Atk: NO_ATTACK, Epoch [1/1], lter [400/468], Loss: 0.28924646973609924
Train. Atk: PGD_ID, Epoch [1/1], lter [100/468], Loss: 1.2768561840057373
Train. Atk: PGD_ID, Epoch [1/1], lter [200/468], Loss: 1.408677101135254
Train. Atk: PGD_ID, Epoch [1/1], lter [300/468], Loss: 1.3168641328811646
Train. Atk: PGD_ID, Epoch [1/1], lter [400/468], Loss: 1.2400099039077759
Model average. Iter: 1/1.


In [ ]:
[ai] = attacked_im
len(ai)

40

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def adv_img_attacked_autoencoder(attacked_im):
  epochs = 5
  outputs = []
  losses = []

  for image in attacked_im:
  
    # Reshaping the image to (-1, 784)
    image = image.reshape(-1, 28*28)
    
    # Output of Autoencoder
    reconstructed = model(image)
    
    # Calculating the loss function
    loss = loss_function(reconstructed, image)
    
    # The gradients are set to zero,
    # the gradient is computed and stored.
    # .step() performs parameter update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Storing the losses in a list for plotting
    losses.append(loss.detach().numpy())
    outputs.append((epochs, image, reconstructed))
  return losses , outputs
  
pgd_auto_encoded_output,pgd_auto_encoded_loss = adv_img_attacked_autoencoder(ai)

RuntimeError: ignored

In [ ]:
adf = df.append(std_acc, ignore_index=True)
df

,NO_ATTACK,RAND_ID,FGSM_ID,PGD_ID,PGD_DCT,PGD_FFT
0,85.38+0.0,52.68+0.0,4.02+0.0,0.1+0.0,0.14+0.0,13.32+0.0
1,69.06+0.0,69.1+0.0,33.4+0.0,35.88+0.0,12.24+0.0,61.28+0.0


In [ ]:
std_acc,attacked_img = get_model_avg(n_models=1, procedure='PGD_DCT')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creati

Train. Atk: NO_ATTACK, Epoch [1/1], lter [100/468], Loss: 0.5165656805038452
Train. Atk: NO_ATTACK, Epoch [1/1], lter [200/468], Loss: 0.43055403232574463
Train. Atk: NO_ATTACK, Epoch [1/1], lter [300/468], Loss: 0.5667160153388977
Train. Atk: NO_ATTACK, Epoch [1/1], lter [400/468], Loss: 0.3694052994251251
Train. Atk: PGD_DCT, Epoch [1/1], lter [100/468], Loss: 1.6326842308044434
Train. Atk: PGD_DCT, Epoch [1/1], lter [200/468], Loss: 1.4029439687728882
Train. Atk: PGD_DCT, Epoch [1/1], lter [300/468], Loss: 1.4316902160644531
Train. Atk: PGD_DCT, Epoch [1/1], lter [400/468], Loss: 1.3228518962860107
Model average. Iter: 1/1.


In [ ]:
df = df.append(std_acc, ignore_index=True)
df

,NO_ATTACK,RAND_ID,FGSM_ID,PGD_ID,PGD_DCT,PGD_FFT
0,85.38+0.0,52.68+0.0,4.02+0.0,0.1+0.0,0.14+0.0,13.32+0.0
1,69.06+0.0,69.1+0.0,33.4+0.0,35.88+0.0,12.24+0.0,61.28+0.0
2,62.58+0.0,63.32+0.0,30.76+0.0,35.7+0.0,32.52+0.0,57.98+0.0


In [ ]:
len(attacked_img)

1

In [ ]:
std_acc,attacked_img = get_model_avg(n_models=1, procedure='PGD_FFT')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creati

Train. Atk: NO_ATTACK, Epoch [1/1], lter [100/468], Loss: 0.4385710656642914
Train. Atk: NO_ATTACK, Epoch [1/1], lter [200/468], Loss: 0.43519142270088196
Train. Atk: NO_ATTACK, Epoch [1/1], lter [300/468], Loss: 0.35225480794906616
Train. Atk: NO_ATTACK, Epoch [1/1], lter [400/468], Loss: 0.3049326241016388
Train. Atk: PGD_FFT, Epoch [1/1], lter [100/468], Loss: 0.5709558725357056
Train. Atk: PGD_FFT, Epoch [1/1], lter [200/468], Loss: 0.7128380537033081
Train. Atk: PGD_FFT, Epoch [1/1], lter [300/468], Loss: 0.6304182410240173
Train. Atk: PGD_FFT, Epoch [1/1], lter [400/468], Loss: 0.48266544938087463
Model average. Iter: 1/1.


In [ ]:
df = df.append(std_acc, ignore_index=True)
df

C:\Users\omen1\AppData\Local\Temp\ipykernel_6924\326607975.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(std_acc, ignore_index=True)


,NO_ATTACK,RAND_ID,FGSM_ID,PGD_ID,PGD_DCT,PGD_FFT
0,87.28+0.0,84.8+0.0,16.96+0.0,3.82+0.0,0.46+0.0,69.68+0.0


In [ ]:
#Generate accuracy df/table
std_acc,attacked_img = get_model_avg(n_models=5, procedure="RR")

In [ ]:
df = df.append(std_acc, ignore_index=True)
df

In [ ]:
#Std accuracy
cost0=train_(model,train_loader,optimizer,loss,atks_model[0])

In [ ]:
df_acc = get_and_add_acc_model(model, test_loader, df_acc,atks_model, model_name="Std")
df_acc

In [ ]:
# Round Robin accuracy
cost_round0 = train_(model_round_rob,train_loader,optimizer_round_rob,loss,atks_model[0])
cost_round1 = round_robin(model_round_rob,train_loader,optimizer_round_rob,loss,atks_model)

In [ ]:
df_acc = get_and_add_acc_model(model_round_rob, test_loader, df_acc, atks_model, model_name="RR")
df_acc

In [ ]:
# Greedy accuracy
check_cuda_avaibilty() 
cost_greedy0 = train_(model_greedy,train_loader,optimizer_greedy,loss,atks_model[0])
cost_greedy1 = greedy(model_greedy,train_loader,valid_loader,optimizer_greedy,loss,atks_model)

In [ ]:
df_acc = get_and_add_acc_model(model_greedy, test_loader, df_acc, atks_model, model_name="Greedy")
df_acc

In [ ]:
#Multi prob. accuracy
check_cuda_avaibilty() 
cost_mult_prob0 = train_(model_mult_prob,train_loader,optimizer_mult_prob,loss,atks_model[0])
cost_mult_prob1 = multivariate_probability(model_mult_prob,train_loader,valid_loader,optimizer_mult_prob,loss,atks_model)

In [ ]:
df_acc = get_and_add_acc_model(model_mult_prob, test_loader, df_acc, atks_model, model_name="MP")
df_acc

In [ ]:
#Initial training in Pixel representation
check_cuda_avaibilty() 
cost_rand0 = train_(model_rand, train_loader, optimizer_rand, loss, atks_model[0])
cost_rand1 = train_(model_rand, train_loader, optimizer_rand, loss, atks_model[1], "RANDOM")

In [ ]:
df_acc = get_and_add_acc_model(model_rand, test_loader, df_acc, atks_model, "Rand")
df_acc

In [ ]:
#Initial training in Pixel representation
check_cuda_avaibilty() 
cost_pgd0 = train_(model_pgd, train_loader, optimizer_pgd, loss)
cost_pgd1 = train_(model_pgd, train_loader, optimizer_pgd, loss, "PGD")

In [ ]:
df_acc = get_and_add_acc_model(model_pgd, test_loader, df_acc, model_name="Pgd")
df_acc


In [ ]:
#Initial training in Pixel representation
check_cuda_avaibilty() 
cost_pgd_dct0 = train_(model_pgd_dct, train_loader, optimizer_pgd_dct, loss)
cost_pgd_dct1 = train_(model_pgd_dct, train_loader, optimizer_pgd_dct, loss, "PGD_DCT")

In [ ]:
df_acc = get_and_add_acc_model(model_pgd_dct, test_loader, df_acc, model_name="Pgd_dct")
df_acc